importing the relevant librariereis to download data and create relevant data frame

In [11]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [12]:
req = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=935851093")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))


neighborhood=pd.DataFrame(df[0])



In [13]:
df1=neighborhood.dropna(subset = ["Borough"])
df1.head(10)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [14]:
df1 = df1.groupby('Postcode').agg({'Neighbourhood':'first', 
                             'Neighbourhood': ', '.join , 'Borough':'first'
                              }).reset_index()
df1.head()


,Postcode,Neighbourhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
3,M1E,"Guildwood, Morningside, West Hill",Scarborough
4,M1G,Woburn,Scarborough


In [15]:
df1.Borough.fillna(df1.Neighbourhood,inplace=True)
df1.head()

,Postcode,Neighbourhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
3,M1E,"Guildwood, Morningside, West Hill",Scarborough
4,M1G,Woburn,Scarborough


In [16]:
df1.count()

Postcode         180
Neighbourhood    180
Borough          180
dtype: int64

to clean the data i have performed foloowing actions-
1)remove rows having nan values in borugh column
2)merging rows having same costal code

downloaded the geospatial data and merged the two dataframes

In [17]:
df = pd.read_csv('http://cocl.us/Geospatial_data')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
dfinal = df.merge(df1, how='inner', left_on='Postal Code', right_on='Postcode')
dfinal.head(10)

,Postal Code,Latitude,Longitude,Postcode,Neighbourhood,Borough
0,M1B,43.806686,-79.194353,M1B,"Rouge, Malvern",Scarborough
1,M1C,43.784535,-79.160497,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,43.763573,-79.188711,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,43.770992,-79.216917,M1G,Woburn,Scarborough
4,M1H,43.773136,-79.239476,M1H,Cedarbrae,Scarborough
5,M1J,43.744734,-79.239476,M1J,Scarborough Village,Scarborough
6,M1K,43.727929,-79.262029,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,43.711112,-79.284577,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,43.716316,-79.239476,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,43.692657,-79.264848,M1N,"Birch Cliff, Cliffside West",Scarborough


importing the relevant libariers for purpose of clustering and making maps

In [22]:
import numpy as np  manner

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 


from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans



! pip install folium
import folium
print('Libraries imported.')

Libraries imported.


In [19]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [24]:

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)



for lat, lng, borough, neighborhood in zip(dfinal['Latitude'], dfinal['Longitude'], dfinal['Borough'], dfinal['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough) 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(Toronto_map)  
    
Toronto_map

In [21]:
toronto_data = dfinal[dfinal['Borough'].str.contains('Toronto')].reset_index(drop = True)


In [25]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [26]:
print(*zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']))

(43.67635739999999, -79.2930312, 'The Beaches') (43.6795571, -79.352188, 'The Danforth West, Riverdale') (43.6689985, -79.31557159999998, 'The Beaches West, India Bazaar') (43.6595255, -79.340923, 'Studio District') (43.7280205, -79.3887901, 'Lawrence Park') (43.7127511, -79.3901975, 'Davisville North') (43.7153834, -79.40567840000001, 'North Toronto West') (43.7043244, -79.3887901, 'Davisville') (43.6895743, -79.38315990000001, 'Moore Park, Summerhill East') (43.68641229999999, -79.4000493, 'Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West') (43.6795626, -79.37752940000001, 'Rosedale') (43.667967, -79.3676753, 'Cabbagetown, St. James Town') (43.6658599, -79.38315990000001, 'Church and Wellesley') (43.6542599, -79.3606359, 'Harbourfront') (43.6571618, -79.37893709999999, 'Ryerson, Garden District') (43.6514939, -79.3754179, 'St. James Town') (43.644770799999996, -79.3733064, 'Berczy Park') (43.6579524, -79.3873826, 'Central Bay Street') (43.65057120000001, -79.3845675,

In [27]:
map_dwtor = folium.Map(location=[latitude, longitude], zoom_start=11)


folium.CircleMarker([latitude, longitude], radius=10, popup=label, color='Red', fill=True, fill_color='Red', fill_opacity=0.9,).add_to(map_dwtor)


for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_dwtor)  
    
map_dwtor

In [28]:
CLIENT_ID = 'CE52W2D4UCAA0KJTKGZS2WKOC04CXC1551PPJJLPM5HXTMCX'      
CLIENT_SECRET = 'DQPSQALZQG5EOVAY1BKH3WF5C352RZB5QZM3FFFXCI02POUU'   
VERSION = '20180901'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CE52W2D4UCAA0KJTKGZS2WKOC04CXC1551PPJJLPM5HXTMCX
CLIENT_SECRET:DQPSQALZQG5EOVAY1BKH3WF5C352RZB5QZM3FFFXCI02POUU


In [29]:
def NearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
radius = 500
LIMIT = 100 
Toronto_venues = NearbyVenues(names = toronto_data['Neighbourhood'], latitudes = toronto_data['Latitude'], longitudes = toronto_data['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [31]:
print(Toronto_venues.shape)
Toronto_venues.head(10)

(1686, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
9,"The Danforth West, Riverdale",43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio


In [32]:

Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,78,78,78,78,78,78
"Chinatown, Grange Park, Kensington Market",81,81,81,81,81,81
Christie,17,17,17,17,17,17


In [33]:
Toronto_venues.groupby('Venue Category').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Afghan Restaurant,1,1,1,1,1,1
Airport,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
Airport Service,3,3,3,3,3,3
Airport Terminal,2,2,2,2,2,2
American Restaurant,23,23,23,23,23,23
Antique Shop,3,3,3,3,3,3


In [37]:
one_hot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [38]:
one_hot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 


fixed_columns = [one_hot.columns[-1]] + list(one_hot.columns[:-1])
one_hot = one_hot[fixed_columns] 

one_hot.head(5)

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [39]:
one_hot.shape

(1686, 234)

In [40]:
torronto_grouped = one_hot.groupby('Neighbourhood').mean().reset_index()
torronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,0.010000,0.0,0.02,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.02,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.01,0.010000,0.0,0.000000,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.02,0.000000,0.070000,0.0,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.010000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.0,0.010000,0.0,0.0,0.00000

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) 
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) 

neighborhoods_venues_sorted = pd.DataFrame(columns=columns) 
neighborhoods_venues_sorted['Neighbourhood'] = torronto_grouped['Neighbourhood']

for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Bar,Thai Restaurant,Bakery,Sushi Restaurant,Breakfast Spot,Pizza Place,Salad Place
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Farmers Market,Beer Bar,Seafood Restaurant,Japanese Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Nightclub,Yoga Studio,Convenience Store,Restaurant,Stadium,Italian Restaurant,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden Center
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Rental Car Location


In [43]:
k = 5

torronto_grouped_clustering = torronto_grouped.drop('Neighbourhood', 1)


kmeans = KMeans(n_clusters=k, random_state=0).fit(torronto_grouped_clustering)


kmeans.labels_[0:38]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       1, 2, 3, 2, 2, 2, 1, 4, 2, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

In [44]:
torronto_merged = toronto_data


torronto_merged['Cluster Labels'] = kmeans.labels_


torronto_merged = torronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

torronto_merged 

,Postal Code,Latitude,Longitude,Postcode,Neighbourhood,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,43.676357,-79.293031,M4E,The Beaches,East Toronto,2,Neighborhood,Trail,Health Food Store,Pub,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M4K,43.679557,-79.352188,M4K,"The Danforth West, Riverdale",East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Brewery,Cosmetics Shop,Bubble Tea Shop,Dessert Shop
2,M4L,43.668999,-79.315572,M4L,"The Beaches West, India Bazaar",East Toronto,2,Park,Pet Store,Pub,Brewery,Sandwich Place,Burrito Place,Restaurant,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop
3,M4M,43.659526,-79.340923,M4M,Studio District,East Toronto,2,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,M4N,43.728020,-79.388790,M4N,Lawrence Park,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
5,M4P,43.712751,-79.390197,M4P,Davisville North,Central Toronto,2,Gym,Food & Drink Shop,Breakfast Spot,Park,Convenience Store,Department Store,Sandwich Place,Hotel,Doner Restaurant,Donut Shop
6,M4R,43.715383,-79.405678,M4R,North Toronto West,Central Toronto,2,Coffee Shop,Clothing Store,Yoga Studio,Salon / Barbershop,Café,Restaurant,Rental Car Location,Chinese Restaurant,Pet Store,Sporting Goods Shop
7,M4S,43.704324,-79.388790,M4S,Davisville,Central Toronto,2,Sandwich Place,Dessert Shop,Toy / Game Store,Café,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Gym,Indian Restaurant
8,M4T,43.689574,-79.383160,M4T,"Moore Park, Summerhill East",Central Toronto,2,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,M4V,43.686412,-79.400049,M4V,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Central Toronto,2,Coffee Shop,Pub,Supermarket,Fried Chicken Joint,Liquor Store,Vietnamese Restaurant,Sushi Restaurant,Pizza Place,Bank,American Restaurant


In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(torronto_merged['Latitude'], torronto_merged['Longitude'], torronto_merged['Neighbourhood'], torronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:


torronto_merged.loc[torronto_merged['Cluster Labels'] == 0, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] #red

,Latitude,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,43.651571,West Toronto,0,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Italian Restaurant,Yoga Studio,Juice Bar,Bank,Bar,Indie Movie Theater


In [47]:

torronto_merged.loc[torronto_merged['Cluster Labels'] == 1, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] 

,Latitude,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,43.657952,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Japanese Restaurant,Burger Joint,Sandwich Place,Thai Restaurant,Gym / Fitness Center,Department Store,Middle Eastern Restaurant,Ice Cream Shop
22,43.711695,Central Toronto,1,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
28,43.646435,Downtown Toronto,1,Coffee Shop,Seafood Restaurant,Restaurant,Café,Hotel,Beer Bar,Japanese Restaurant,Cocktail Bar,Gym,Farmers Market


In [48]:


torronto_merged.loc[torronto_merged['Cluster Labels'] == 2, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] 

,Latitude,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.676357,East Toronto,2,Neighborhood,Trail,Health Food Store,Pub,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,43.679557,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Brewery,Cosmetics Shop,Bubble Tea Shop,Dessert Shop
2,43.668999,East Toronto,2,Park,Pet Store,Pub,Brewery,Sandwich Place,Burrito Place,Restaurant,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop
3,43.659526,East Toronto,2,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,43.728020,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
5,43.712751,Central Toronto,2,Gym,Food & Drink Shop,Breakfast Spot,Park,Convenience Store,Department Store,Sandwich Place,Hotel,Doner Restaurant,Donut Shop
6,43.715383,Central Toronto,2,Coffee Shop,Clothing Store,Yoga Studio,Salon / Barbershop,Café,Restaurant,Rental Car Location,Chinese Restaurant,Pet Store,Sporting Goods Shop
7,43.704324,Central Toronto,2,Sandwich Place,Dessert Shop,Toy / Game Store,Café,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Gym,Indian Restaurant
8,43.689574,Central Toronto,2,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,43.686412,Central Toronto,2,Coffee Shop,Pub,Supermarket,Fried Chicken Joint,Liquor Store,Vietnamese Restaurant,Sushi Restaurant,Pizza Place,Bank,American Restaurant


In [49]:
# Cluster 3
# Airport services 
torronto_merged.loc[torronto_merged['Cluster Labels'] == 3, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] #green

,Latitude,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,43.67271,Central Toronto,3,Café,Coffee Shop,Sandwich Place,Indian Restaurant,Pub,Liquor Store,BBQ Joint,Donut Shop,History Museum,Middle Eastern Restaurant


In [50]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 4, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]] #green

,Latitude,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,43.648429,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Asian Restaurant,Japanese Restaurant,Gym,American Restaurant,Gastropub,Steakhouse,Hotel
